<a href="https://colab.research.google.com/github/IKeeso/Project9_data/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stanza
!pip install conllu
!git clone https://github.com/IKeeso/Project9_data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
import stanza
from stanza.utils.conll import CoNLL
from conllu import parse
from conllu import TokenList
import re

!ls Project9_data


expressen.txt  Project.ipynb


In [3]:
stanza.download('sv')
nlp = stanza.Pipeline(lang='sv')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |
| ner       | suc3shuffle        |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [4]:
def tag_and_convert(filename):
  %cd /content/Project9_data
  with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()
    doc = nlp(text)
    file.close()
    pathname = filename.split(".")[0]
    conllufilename = f"{pathname}.conllu"
  CoNLL.write_doc2conll(doc, conllufilename)

In [5]:
tag_and_convert("expressen.txt")

/content/Project9_data


In [6]:
"""
!git remote -v
!git init
!git config --global user.email 'isali352@student.liu.se'
!git config --global user.name 'IKeeso'
!git add expressen.conllu
!git commit -m 'New commit'  # commit in Colab
!git push origin main
"""

"\n!git remote -v\n!git init\n!git config --global user.email 'isali352@student.liu.se'\n!git config --global user.name 'IKeeso'\n!git add expressen.conllu\n!git commit -m 'New commit'  # commit in Colab\n!git push origin main\n"

In [27]:
class SemanticAnalyzer:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = self.parse_conll_u()
        self.identity_first_search_string = re.compile(r"^.*\baut(ism|istisk|istiska|istiske)\s[A-Ö]*\s", re.IGNORECASE)
        self.person_first_search_string = "(svensk|svenskar|ungdom|ungdomar|vuxen|vuxna|amerikan|amerikaner|amputee|amputerade|astmatiker|astmatiker|aussie|aussies|australisk|australier|bankir|bankirer|chef|chefer|pojke|pojkar|britt|britter|canberran|canberrans|barn|barn|medborgare|medborgare|klient|klienter|tävlande|tävlande|kund|kunder|pappa|pappor|dotter|döttrar|diabetiker|diabetiker|dietist|dietister|förare|förare|anställd|anställda|fan|fans|far|fäder|fransman|fransmän|franska kvinnor|vän|vänner|flicka|flickor|kille|killar|individ|individer|barn|barn|damer|dam|man|män|modell|modeller|mor|mödrar|bilist|bilister|mamma|mammor|kompis|kompisar|förälder|föräldrar|deltagare|deltagare|passagerare|passagerare|patient|patienter|människor|person|personer|personer|förskolebarn|förskolebarn|queenslander|queenslanders|boende|boende|rökare|rökare|socialiteter|socialites|soldat|soldater|son|söner|student|studenter|ämne|ämnen|tasmanian|tasmanianer|tonåring|tonåringar|tonåringar|tonåringar|tonåringar|resenär|resenärer|victorian|victorians|volontär|volontärer|kvinna|kvinnor|arbetare|arbetare|ungdom|ungdomar) med autism"
        self.sentence_matches = []

    def parse_conll_u(self):
      with open(file_path, 'r', encoding='utf-8') as file:
        data = {}
        sentence = []
        for line in file:
          line = line.strip()
          if line.startswith('# text'):
            sentence = []
          if line.startswith("# sent_id"):
            line = line.split("=")
            key = line[1].strip()
            sentence = []
          elif line == '':
            if sentence:
              data[key] = sentence
          else:
              sentence.append(line.split('\t'))
      return data

    def regx_analyze_sentence(self, sent_id, sentence, search_string):
      words = ' '.join(token[1] for token in sentence)
      if search_string.match(words):
        self.sentence_matches.append(sentence)
        return True
      else:
        return False

# Example usage:
file_path = 'expressen.conllu'
analyzer = SemanticAnalyzer(file_path)

for id, sentence in analyzer.data.items():
  if analyzer.regx_analyze_sentence(id, sentence, analyzer.identity_first_search_string):
    print(f"(sent_id: {id})", ' '.join(token[1] for token in sentence), "\n")

(sent_id: 0) ” Det ser otroligt olika ut för olika autistiska personer ” 

(sent_id: 4) Många med autism känner sig missförstådda och har svårt att klara skolan . 

(sent_id: 5) Det vill föreningen Autism Sverige ändra på . 

(sent_id: 6) – Varannan elev med autism når inte målen i skolan , på högstadiet är andelen ännu större . 

(sent_id: 7) Vi behöver en skola som ger alla elever möjlighet att lyckas utifrån sina egna förutsättningar , säger Linn Rivero Mildén som är kommunikatör på Autism Sverige . 

(sent_id: 8) Det är viktigt att öka förståelsen för att olika personer med autism har olika förutsättningar , menar Linn . 

(sent_id: 9) – Det ser otroligt olika ut för olika autistiska personer och spektrumet kan vara stort i en och samma person , säger hon . 

(sent_id: 10) Sandra Dahlberg tillsammans med Gunilla Sundblad , förbundssekreterare på Autism Sverige . 

(sent_id: 11) Förändrade attityder krävs för att skapa ett samhälle för alla Sedan 2018 har Postkodlotteriet stöttat Au